### Clean up first

In [1]:
import sys.process._
"rm -rf /tmp/hudicow/".!!
"mkdir -p /tmp/hudicow".!!
lazy val result3 = "tree -a /tmp/hudicow/".!!
println(result3)


/tmp/hudicow/

0 directories, 0 files



import sys.process._
res1_1: String = ""
res1_2: String = ""
result3: String = [lazy]

### Setup project

In [2]:
import $ivy.`org.apache.hudi:hudi-spark3.3-bundle_2.12:1.0.0`
// import $ivy.`org.apache.hudi:hudi-common:1.0.0`
import $ivy.`org.apache.spark:spark-sql_2.12:3.3.2`
import $ivy.`org.apache.spark:spark-avro_2.12:3.3.2`
import $ivy.`org.apache.spark:spark-hive_2.12:3.3.2`
import $cp.`CustomMergeIntoConnector.jar`

import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.hudi.QuickstartUtils._
import org.apache.spark.sql.hive.HiveExternalCatalog
import scala.collection.JavaConversions._
import org.apache.spark.sql.SaveMode._
import org.apache.hudi.DataSourceReadOptions._
import org.apache.hudi.DataSourceWriteOptions._
import org.apache.hudi.common.table.HoodieTableConfig._
import org.apache.hudi.config.HoodieWriteConfig._
import org.apache.hudi.keygen.constant.KeyGeneratorOptions._
import org.apache.hudi.common.model.HoodieRecord
import com.payloads.CustomMergeIntoConnector

val spark = SparkSession.builder()
  .appName("HudiLocalSession")
  .master("local[1]")  // Runs on local machine with 1 local worker / thread, so we can see the impact on COPY_ON_WRITE
  .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
  .config("spark.sql.extensions", "org.apache.spark.sql.hudi.HoodieSparkSessionExtension")
  .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.hudi.catalog.HoodieCatalog")
  .config("spark.kryo.registrator", "org.apache.spark.HoodieSparkKryoRegistrar")
  .config("spark.driver.extraJavaOptions", "-Dscala.repl.maxprintstring=0")
  .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
  
// http://localhost:4040/jobs/
println("Spark with Hudi is ready!")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties


25/03/05 20:28:29 WARN Utils: Your hostname, DESKTOP-M94RUSC resolves to a loopback address: 127.0.1.1; using 172.17.75.227 instead (on interface eth0)
25/03/05 20:28:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/03/05 20:28:30 INFO SparkContext: Running Spark version 3.3.2
25/03/05 20:28:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/05 20:28:30 INFO ResourceUtils: ==============================================================
25/03/05 20:28:30 INFO ResourceUtils: No custom resources configured for spark.driver.
25/03/05 20:28:30 INFO ResourceUtils: ==============================================================
25/03/05 20:28:30 INFO SparkContext: Submitted application: HudiLocalSession
25/03/05 20:28:30 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024

import $ivy.$
import $ivy.$
import $ivy.$
import $ivy.$
import $cp.$
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.hudi.QuickstartUtils._
import org.apache.spark.sql.hive.HiveExternalCatalog
import scala.collection.JavaConversions._
import org.apache.spark.sql.SaveMode._
import org.apache.hudi.DataSourceReadOptions._
import org.apache.hudi.DataSourceWriteOptions._
import org.apache.hudi.common.table.HoodieTableConfig._
import org.apache.hudi.config.HoodieWriteConfig._
import org.apache.hudi.keygen.constant.KeyGeneratorOptions._
import org.apache.hudi.common.model.HoodieRecord
import com.payloads.CustomMergeIntoConnector
spark: SparkSession = org.apache.spark.sql.SparkSession@de02d9e

In [3]:
lazy val result = "tree -a /tmp/hudicow/".!!
println(result)

/tmp/hudicow/

0 directories, 0 files



result: String = [lazy]

### Create and Insert Table

In [4]:
val columns = Seq("ts","uuid","rider","driver","fare","city")
val data =
  Seq((1695159649087L,"334e26e9-8355-45cc-97c6-c31daf0df330","rider-A","driver-K",19.10,"san_francisco"),
    (1695091554788L,"e96c4396-3fad-413a-a942-4cb36106d721","rider-C","driver-M",27.70 ,"san_francisco"),
    (1695046462179L,"9909a8b1-2d15-4d3d-8ec9-efc48c536a00","rider-D","driver-L",33.90 ,"san_francisco"),
    (1695516137016L,"e3cf430c-889d-4015-bc98-59bdce1e530c","rider-F","driver-P",34.15,"sao_paulo"    ),
    (1695115999911L,"c8abbe79-8d89-47ea-b4ce-4d224bae5bfa","rider-J","driver-T",17.85,"chennai"));

val tableName = "trips_table"
val basePath = "file:///tmp/hudicow/trips_table"
var inserts = spark.createDataFrame(data).toDF(columns:_*)

inserts.write.format("hudi").
  option("hoodie.datasource.write.partitionpath.field", "city").
  option("hoodie.datasource.write.hive_style_partitioning", "true"). // Force hive style dir naming for partitions.
  option("hoodie.datasource.write.storage.type", "COPY_ON_WRITE").
  option("hoodie.table.name", tableName).
  mode(Overwrite).
  save(basePath)



# WARNING: Unable to get Instrumentation. Dynamic Attach failed. You may add this JAR as -javaagent manually, or supply -Djdk.attach.allowAttachSelf
# WARNING: Unable to attach Serviceability Agent. Unable to attach even with module exceptions: [org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed., org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed., org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed.]


columns: Seq[String] = List("ts", "uuid", "rider", "driver", "fare", "city")
data: Seq[(Long, String, String, String, Double, String)] = List(
  (
    1695159649087L,
    "334e26e9-8355-45cc-97c6-c31daf0df330",
    "rider-A",
    "driver-K",
    19.1,
    "san_francisco"
  ),
  (
    1695091554788L,
    "e96c4396-3fad-413a-a942-4cb36106d721",
    "rider-C",
    "driver-M",
    27.7,
    "san_francisco"
  ),
  (
    1695046462179L,
    "9909a8b1-2d15-4d3d-8ec9-efc48c536a00",
    "rider-D",
    "driver-L",
    33.9,
    "san_francisco"
  ),
  (
    1695516137016L,
    "e3cf430c-889d-4015-bc98-59bdce1e530c",
    "rider-F",
    "driver-P",
    34.15,
    "sao_paulo"
  ),
  (
    1695115999911L,
    "c8abbe79-8d89-47ea-b4ce-4d224bae5bfa",
    "rider-J",
    "driver-T",
    17.85,
...
tableName: String = "trips_table"
basePath: String = "file:///tmp/hudicow/trips_table"
inserts: DataFrame = [ts: bigint, uuid: string ... 4 more fields]

In [5]:
lazy val result = "tree -a /tmp/hudicow/".!!
println(result)

/tmp/hudicow/
└── trips_table
    ├── .hoodie
    │   ├── .aux
    │   │   └── .bootstrap
    │   │       ├── .fileids
    │   │       └── .partitions
    │   ├── .hoodie.properties.crc
    │   ├── .schema
    │   ├── .temp
    │   ├── hoodie.properties
    │   ├── metadata
    │   │   ├── .hoodie
    │   │   │   ├── .aux
    │   │   │   │   └── .bootstrap
    │   │   │   │       ├── .fileids
    │   │   │   │       └── .partitions
    │   │   │   ├── .hoodie.properties.crc
    │   │   │   ├── .schema
    │   │   │   ├── .temp
    │   │   │   ├── hoodie.properties
    │   │   │   └── timeline
    │   │   │       ├── .00000000000000000.deltacommit.inflight.crc
    │   │   │       ├── .00000000000000000.deltacommit.requested.crc
    │   │   │       ├── .00000000000000000_20250305202834701.deltacommit.crc
    │   │   │       ├── .20250305202833534.deltacommit.inflight.crc
    │   │   │       ├── .20250305202833534.deltacommit.requested.crc
    │   │   │       ├── .20250305202833534_202503

result: String = [lazy]

* If we check the Hudi table dir, we can find several parquet files.
* Here we have 1 parquet file for each city/partition.
* This is because we set the number of worker to 1 and the file is tiny.

In [6]:
val command = "find /tmp/hudicow/trips_table -name \\\"*.parquet\\\" -exec stat -c \\\"%y %s %n\\\" {} + | sort"
lazy val result = Seq("bash", "-c", s"eval $command").!!
println(result)

2025-03-05 20:28:36.456832582 +0900 435948 /tmp/hudicow/trips_table/city=sao_paulo/ce80064b-d0eb-474f-adf5-5e18721902da-1_0-6-0_20250305202833534.parquet
2025-03-05 20:28:36.456832582 +0900 436116 /tmp/hudicow/trips_table/city=san_francisco/ce80064b-d0eb-474f-adf5-5e18721902da-0_0-6-0_20250305202833534.parquet
2025-03-05 20:28:36.466840530 +0900 435920 /tmp/hudicow/trips_table/city=chennai/ce80064b-d0eb-474f-adf5-5e18721902da-2_0-6-0_20250305202833534.parquet



command: String = "find /tmp/hudicow/trips_table -name \\\"*.parquet\\\" -exec stat -c \\\"%y %s %n\\\" {} + | sort"
result: String = [lazy]

* The timeline dir has .commit files on CoW.

In [7]:
val command = """find /tmp/hudicow/trips_table/.hoodie  -name "*commit" -exec stat -c \"%y %s %n\" {} + | sort"""
lazy val result = Seq("bash", "-c", s"eval $command").!!
println(result)

2025-03-05 20:28:34.697952354 +0900 9913 /tmp/hudicow/trips_table/.hoodie/metadata/.hoodie/timeline/00000000000000000_20250305202834701.deltacommit
2025-03-05 20:28:36.737055030 +0900 9985 /tmp/hudicow/trips_table/.hoodie/metadata/.hoodie/timeline/20250305202833534_20250305202836740.deltacommit
2025-03-05 20:28:36.767078864 +0900 3584 /tmp/hudicow/trips_table/.hoodie/timeline/20250305202833534_20250305202836772.commit



command: String = "find /tmp/hudicow/trips_table/.hoodie  -name \"*commit\" -exec stat -c \\\"%y %s %n\\\" {} + | sort"
result: String = [lazy]

### Query data

In [8]:
val tripsDF = spark.read.format("hudi").load(basePath)
tripsDF.createOrReplaceTempView("trips_table")

spark.sql("SELECT uuid, fare, ts, rider, driver, city FROM  trips_table WHERE fare > 20.0").show()
spark.sql("SELECT _hoodie_commit_time, _hoodie_record_key, _hoodie_partition_path, rider, driver, fare FROM  trips_table").show()

+--------------------+-----+-------------+-------+--------+-------------+
|                uuid| fare|           ts|  rider|  driver|         city|
+--------------------+-----+-------------+-------+--------+-------------+
|e96c4396-3fad-413...| 27.7|1695091554788|rider-C|driver-M|san_francisco|
|9909a8b1-2d15-4d3...| 33.9|1695046462179|rider-D|driver-L|san_francisco|
|e3cf430c-889d-401...|34.15|1695516137016|rider-F|driver-P|    sao_paulo|
+--------------------+-----+-------------+-------+--------+-------------+

+-------------------+--------------------+----------------------+-------+--------+-----+
|_hoodie_commit_time|  _hoodie_record_key|_hoodie_partition_path|  rider|  driver| fare|
+-------------------+--------------------+----------------------+-------+--------+-----+
|  20250305202833534|20250305202833534...|    city=san_francisco|rider-A|driver-K| 19.1|
|  20250305202833534|20250305202833534...|    city=san_francisco|rider-C|driver-M| 27.7|
|  20250305202833534|202503052028335

tripsDF: DataFrame = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 9 more fields]

In [9]:
val command = "find /tmp/hudicow/trips_table -name \\\"*.parquet\\\" -exec stat -c \\\"%y %s %n\\\" {} + | sort"
lazy val result = Seq("bash", "-c", s"eval $command").!!
println(result)
val command2 = """find /tmp/hudicow/trips_table/.hoodie  -name "*commit" -exec stat -c \"%y %s %n\" {} + | sort"""
lazy val result2 = Seq("bash", "-c", s"eval $command2").!!
println(result2)

2025-03-05 20:28:36.456832582 +0900 435948 /tmp/hudicow/trips_table/city=sao_paulo/ce80064b-d0eb-474f-adf5-5e18721902da-1_0-6-0_20250305202833534.parquet
2025-03-05 20:28:36.456832582 +0900 436116 /tmp/hudicow/trips_table/city=san_francisco/ce80064b-d0eb-474f-adf5-5e18721902da-0_0-6-0_20250305202833534.parquet
2025-03-05 20:28:36.466840530 +0900 435920 /tmp/hudicow/trips_table/city=chennai/ce80064b-d0eb-474f-adf5-5e18721902da-2_0-6-0_20250305202833534.parquet

2025-03-05 20:28:34.697952354 +0900 9913 /tmp/hudicow/trips_table/.hoodie/metadata/.hoodie/timeline/00000000000000000_20250305202834701.deltacommit
2025-03-05 20:28:36.737055030 +0900 9985 /tmp/hudicow/trips_table/.hoodie/metadata/.hoodie/timeline/20250305202833534_20250305202836740.deltacommit
2025-03-05 20:28:36.767078864 +0900 3584 /tmp/hudicow/trips_table/.hoodie/timeline/20250305202833534_20250305202836772.commit



command: String = "find /tmp/hudicow/trips_table -name \\\"*.parquet\\\" -exec stat -c \\\"%y %s %n\\\" {} + | sort"
result: String = [lazy]
command2: String = "find /tmp/hudicow/trips_table/.hoodie  -name \"*commit\" -exec stat -c \\\"%y %s %n\\\" {} + | sort"
result2: String = [lazy]

### Update data

In [10]:
// Lets read data from target Hudi table, modify fare column for rider-D and update it. 
import spark.implicits._
val updatesDf = spark.read.format("hudi").load(basePath).filter($"rider" === "rider-D").withColumn("fare", col("fare") * 10)

updatesDf.write.format("hudi").
  option("hoodie.datasource.write.operation", "upsert").
  option("hoodie.datasource.write.partitionpath.field", "city").
  option("hoodie.table.name", tableName).
  mode(Append).
  save(basePath)

import spark.implicits._
updatesDf: DataFrame = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 9 more fields]

* Notice after the update, we have 2 commit files and 3 deltacommit files (1 is initial state for table creation, labeled all 0).

In [11]:
lazy val result = "tree -a /tmp/hudicow/trips_table".!!
println(result)

/tmp/hudicow/trips_table
├── .hoodie
│   ├── .aux
│   │   └── .bootstrap
│   │       ├── .fileids
│   │       └── .partitions
│   ├── .hoodie.properties.crc
│   ├── .schema
│   ├── .temp
│   ├── hoodie.properties
│   ├── metadata
│   │   ├── .hoodie
│   │   │   ├── .aux
│   │   │   │   └── .bootstrap
│   │   │   │       ├── .fileids
│   │   │   │       └── .partitions
│   │   │   ├── .hoodie.properties.crc
│   │   │   ├── .schema
│   │   │   ├── .temp
│   │   │   ├── hoodie.properties
│   │   │   └── timeline
│   │   │       ├── .00000000000000000.deltacommit.inflight.crc
│   │   │       ├── .00000000000000000.deltacommit.requested.crc
│   │   │       ├── .00000000000000000_20250305202834701.deltacommit.crc
│   │   │       ├── .20250305202833534.deltacommit.inflight.crc
│   │   │       ├── .20250305202833534.deltacommit.requested.crc
│   │   │       ├── .20250305202833534_20250305202836740.deltacommit.crc
│   │   │       ├── .20250305202839238.deltacommit.inflight.crc
│   │   │       ├

result: String = [lazy]

* Lets observe the parquet files in the update.
* Note the original has smaller timestamp and newer one has larger timestamp.

In [12]:
val command = """find /tmp/hudicow/trips_table -name "*.parquet" -exec stat -c \"%y %s %n\" {} + | sort"""
lazy val result = Seq("bash", "-c", s"eval $command").!!
println(result)

2025-03-05 20:28:36.456832582 +0900 435948 /tmp/hudicow/trips_table/city=sao_paulo/ce80064b-d0eb-474f-adf5-5e18721902da-1_0-6-0_20250305202833534.parquet
2025-03-05 20:28:36.456832582 +0900 436116 /tmp/hudicow/trips_table/city=san_francisco/ce80064b-d0eb-474f-adf5-5e18721902da-0_0-6-0_20250305202833534.parquet
2025-03-05 20:28:36.466840530 +0900 435920 /tmp/hudicow/trips_table/city=chennai/ce80064b-d0eb-474f-adf5-5e18721902da-2_0-6-0_20250305202833534.parquet
2025-03-05 20:28:39.815689984 +0900 436091 /tmp/hudicow/trips_table/city=san_francisco/ce80064b-d0eb-474f-adf5-5e18721902da-0_0-20-22_20250305202839238.parquet



command: String = "find /tmp/hudicow/trips_table -name \"*.parquet\" -exec stat -c \\\"%y %s %n\\\" {} + | sort"
result: String = [lazy]

#### Observing updated partition (san_francisco)
* With COPY_ON_WRITE (CoW), modified row forces the recreation of the affected file (not the whole partition).
* In our case we have only 1 file, so we recreate the entire partition.
* If we have more than 1 file, only the affected file is recreated (try with local[*]).
* This means, for every write, Hudi must find the affected file and rewrite the entire file.
    * This is why CoW is more expensive than MERGE_ON_READ for writes.

In [13]:
val command = """find /tmp/hudicow/trips_table -path "*san_francisco*" -name "*.parquet" -exec stat -c \"%y %s %n\" {} + | sort"""
lazy val result = Seq("bash", "-c", s"eval $command").!!
lazy val parquetPaths = result.split("\n")
for(parquetPath <- parquetPaths){
    val path = parquetPath.split(" ")(4)
    println(parquetPath)
    spark.read.format("parquet").load(path).show(truncate=false)
    println()
}

2025-03-05 20:28:36.456832582 +0900 436116 /tmp/hudicow/trips_table/city=san_francisco/ce80064b-d0eb-474f-adf5-5e18721902da-0_0-6-0_20250305202833534.parquet
+-------------------+---------------------+---------------------+----------------------+----------------------------------------------------------------------+-------------+------------------------------------+-------+--------+----+-------------+
|_hoodie_commit_time|_hoodie_commit_seqno |_hoodie_record_key   |_hoodie_partition_path|_hoodie_file_name                                                     |ts           |uuid                                |rider  |driver  |fare|city         |
+-------------------+---------------------+---------------------+----------------------+----------------------------------------------------------------------+-------------+------------------------------------+-------+--------+----+-------------+
|20250305202833534  |20250305202833534_0_1|20250305202833534_0_0|city=san_francisco    |ce80064b-d0eb

command: String = "find /tmp/hudicow/trips_table -path \"*san_francisco*\" -name \"*.parquet\" -exec stat -c \\\"%y %s %n\\\" {} + | sort"
result: String = [lazy]
parquetPaths: Array[String] = [lazy]

In [14]:
val command = """find /tmp/hudicow/trips_table/.hoodie  -name "*commit" -exec stat -c \"%y %s %n\" {} + | sort"""
lazy val result = Seq("bash", "-c", s"eval $command").!!
println(result)

2025-03-05 20:28:34.697952354 +0900 9913 /tmp/hudicow/trips_table/.hoodie/metadata/.hoodie/timeline/00000000000000000_20250305202834701.deltacommit
2025-03-05 20:28:36.737055030 +0900 9985 /tmp/hudicow/trips_table/.hoodie/metadata/.hoodie/timeline/20250305202833534_20250305202836740.deltacommit
2025-03-05 20:28:36.767078864 +0900 3584 /tmp/hudicow/trips_table/.hoodie/timeline/20250305202833534_20250305202836772.commit
2025-03-05 20:28:40.106949782 +0900 9985 /tmp/hudicow/trips_table/.hoodie/metadata/.hoodie/timeline/20250305202839238_20250305202840110.deltacommit
2025-03-05 20:28:40.127047966 +0900 3024 /tmp/hudicow/trips_table/.hoodie/timeline/20250305202839238_20250305202840134.commit



command: String = "find /tmp/hudicow/trips_table/.hoodie  -name \"*commit\" -exec stat -c \\\"%y %s %n\\\" {} + | sort"
result: String = [lazy]

#### Inspecting commit files.
* There are two commit files: deltacommit and commit.
    * deltacommit contains the partition write stats (files and their metadata).
    * commit contains the updated partitions and their files plus schema for the file.
* https://github.com/apache/hudi/issues/2288
    * Each file is created with extra extensions instead of renamed to improve compatibility with different DFSs.
    * Might be an issue for some DFS because it might creates too many tiny files.
    * Can be expensive for metadata server.

In [15]:
val command = """find /tmp/hudicow/trips_table -name "*.deltacommit" -exec stat -c \"%y %s %n\" {} + | sort"""
lazy val result = Seq("bash", "-c", s"eval $command").!!
lazy val hudiMetas = result.split("\n")
for(hudiMeta <- hudiMetas){
    val path = hudiMeta.split(" ")(4)
    println(path)
    spark.read.format("avro").load(path).select("operationType", "partitionToWriteStats", "extraMetadata", "version").show(truncate=false)
    println()
}

/tmp/hudicow/trips_table/.hoodie/metadata/.hoodie/timeline/00000000000000000_20250305202834701.deltacommit
+-------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

command: String = "find /tmp/hudicow/trips_table -name \"*.deltacommit\" -exec stat -c \\\"%y %s %n\\\" {} + | sort"
result: String = [lazy]
hudiMetas: Array[String] = [lazy]

In [16]:
val command = """find /tmp/hudicow/trips_table -name "*.commit" -exec stat -c \"%y %s %n\" {} + | sort"""
lazy val result = Seq("bash", "-c", s"eval $command").!!
lazy val hudiMetas = result.split("\n")
for(hudiMeta <- hudiMetas){
    val path = hudiMeta.split(" ")(4)
    println(path)
    spark.read.format("avro").load(path).select("operationType", "partitionToWriteStats", "extraMetadata", "version").show(truncate=false)
    println()
}

/tmp/hudicow/trips_table/.hoodie/timeline/20250305202833534_20250305202836772.commit
+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------

command: String = "find /tmp/hudicow/trips_table -name \"*.commit\" -exec stat -c \\\"%y %s %n\\\" {} + | sort"
result: String = [lazy]
hudiMetas: Array[String] = [lazy]

* Note that in deltacommit, the initial 00000000 file is BULK_INSERT and the other insertion are UPSERT_PREPPED.

In [17]:
val hudiMetadata2 = "/tmp/hudicow/trips_table/.hoodie/timeline/*.commit"
val dfHudiMeta2 = spark.read.format("avro").load(hudiMetadata2)
dfHudiMeta2.select("operationType", "partitionToWriteStats", "extraMetadata", "version").show(truncate=false)

+--------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

hudiMetadata2: String = "/tmp/hudicow/trips_table/.hoodie/timeline/*.commit"
dfHudiMeta2: DataFrame = [partitionToWriteStats: map<string,array<struct<fileId:string,path:string,prevCommit:string,numWrites:bigint,numDeletes:bigint,numUpdateWrites:bigint,totalWriteBytes:bigint,totalWriteErrors:bigint,partitionPath:string,totalLogRecords:bigint,totalLogFiles:bigint,totalUpdatedRecordsCompacted:bigint,numInserts:bigint,totalLogBlocks:bigint,totalCorruptLogBlock:bigint,totalRollbackBlocks:bigint,fileSizeInBytes:bigint,logVersion:int,logOffset:bigint,baseFile:string,logFiles:array<string>,cdcStats:map<string,bigint>>>>, extraMetadata: map<string,string> ... 2 more fields]

* Note that in commit, there is no 0000000000000... file.
* The initial write is BULK_INSERT.
* The update is UPSERT_PREPPED.

In [18]:
val tripsDF = spark.read.format("hudi").load(basePath)
tripsDF.createOrReplaceTempView("trips_table")
spark.sql(s"SELECT _hoodie_commit_time, fare, rider FROM trips_table;").show()

+-------------------+-----+-------+
|_hoodie_commit_time| fare|  rider|
+-------------------+-----+-------+
|  20250305202833534| 19.1|rider-A|
|  20250305202833534| 27.7|rider-C|
|  20250305202839238|339.0|rider-D|
|  20250305202833534|34.15|rider-F|
|  20250305202833534|17.85|rider-J|
+-------------------+-----+-------+



tripsDF: DataFrame = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 9 more fields]

### Merging Data

In [19]:
val adjustedFareDF = spark.read.format("hudi").
  load(basePath).limit(2).
  withColumn("fare", col("fare") * 10)

adjustedFareDF.write.format("hudi").
  option("hoodie.datasource.write.payload.class","com.payloads.CustomMergeIntoConnector").
  mode(Append).
  save(basePath)
// Notice Fare column has been updated but all other columns remain intact.
spark.read.format("hudi").load(basePath).show()

+-------------------+--------------------+--------------------+----------------------+--------------------+-------------+--------------------+-------+--------+-----+-------------+
|_hoodie_commit_time|_hoodie_commit_seqno|  _hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|           ts|                uuid|  rider|  driver| fare|         city|
+-------------------+--------------------+--------------------+----------------------+--------------------+-------------+--------------------+-------+--------+-----+-------------+
|  20250305202842976|20250305202842976...|20250305202833534...|    city=san_francisco|ce80064b-d0eb-474...|1695159649087|334e26e9-8355-45c...|rider-A|driver-K|210.1|san_francisco|
|  20250305202842976|20250305202842976...|20250305202833534...|    city=san_francisco|ce80064b-d0eb-474...|1695091554788|e96c4396-3fad-413...|rider-C|driver-M|304.7|san_francisco|
|  20250305202839238|20250305202839238...|20250305202833534...|    city=san_francisco|ce80064b-d0eb-

adjustedFareDF: DataFrame = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 9 more fields]

* For rider-A, the fare previously was 19.1, now its 210.1.
    * In the adjustedFareDF, the fare is 10*fare, so adjustedFare is 191.
    * In the CustomMergeIntoConnector, the final fare is the original fare + adjustedFare, which is 19.1 + 191, thus it is 210.1.
    * The same applies for rider-D.
* If you notice, with 1 update and 1 merge upsert, now we have 3 parquet files for partition san_francisco.
* Next lets check the parquet files for san_francisco.
    * We can see the content represents the state of the file for the commit time.
    * I think older commits will be consolidated later.
    * https://hudi.apache.org/docs/cleaning

In [20]:
val command = """find /tmp/hudicow/trips_table -path "*san_francisco*" -name "*.parquet" -exec stat -c \"%y %s %n\" {} + | sort"""
lazy val result = Seq("bash", "-c", s"eval $command").!!
lazy val parquetFiles = result.split("\n")
for(parquetFile <- parquetFiles){
    val path = parquetFile.split(" ")(4)
    println(path)
    spark.read.format("parquet").load(path).show(truncate=false)
    println()
}

/tmp/hudicow/trips_table/city=san_francisco/ce80064b-d0eb-474f-adf5-5e18721902da-0_0-6-0_20250305202833534.parquet
+-------------------+---------------------+---------------------+----------------------+----------------------------------------------------------------------+-------------+------------------------------------+-------+--------+----+-------------+
|_hoodie_commit_time|_hoodie_commit_seqno |_hoodie_record_key   |_hoodie_partition_path|_hoodie_file_name                                                     |ts           |uuid                                |rider  |driver  |fare|city         |
+-------------------+---------------------+---------------------+----------------------+----------------------------------------------------------------------+-------------+------------------------------------+-------+--------+----+-------------+
|20250305202833534  |20250305202833534_0_1|20250305202833534_0_0|city=san_francisco    |ce80064b-d0eb-474f-adf5-5e18721902da-0_0-6-0_20250305202

command: String = "find /tmp/hudicow/trips_table -path \"*san_francisco*\" -name \"*.parquet\" -exec stat -c \\\"%y %s %n\\\" {} + | sort"
result: String = [lazy]
parquetFiles: Array[String] = [lazy]

In [21]:
val command = """find /tmp/hudicow/trips_table -name "*.deltacommit" -exec stat -c \"%y %s %n\" {} + | sort"""
lazy val result = Seq("bash", "-c", s"eval $command").!!
lazy val hudiMetas = result.split("\n")
for(hudiMeta <- hudiMetas){
    val path = hudiMeta.split(" ")(4)
    println(path)
    spark.read.format("avro").load(path).select("operationType", "partitionToWriteStats", "extraMetadata", "version").show(truncate=false)
    println()
}

/tmp/hudicow/trips_table/.hoodie/metadata/.hoodie/timeline/00000000000000000_20250305202834701.deltacommit
+-------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

command: String = "find /tmp/hudicow/trips_table -name \"*.deltacommit\" -exec stat -c \\\"%y %s %n\\\" {} + | sort"
result: String = [lazy]
hudiMetas: Array[String] = [lazy]

In [22]:
val command = """find /tmp/hudicow/trips_table -name "*.commit" -exec stat -c \"%y %s %n\" {} + | sort"""
lazy val result = Seq("bash", "-c", s"eval $command").!!
lazy val hudiMetas = result.split("\n")
for(hudiMeta <- hudiMetas){
    val path = hudiMeta.split(" ")(4)
    println(path)
    spark.read.format("avro").load(path).select("operationType", "partitionToWriteStats", "extraMetadata", "version").show(truncate=false)
    println()
}

/tmp/hudicow/trips_table/.hoodie/timeline/20250305202833534_20250305202836772.commit
+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------

command: String = "find /tmp/hudicow/trips_table -name \"*.commit\" -exec stat -c \\\"%y %s %n\\\" {} + | sort"
result: String = [lazy]
hudiMetas: Array[String] = [lazy]

### Merging Data (Partial Updates) if use sql and not CustomMergeIntoConnector.
* Reason for trips_table2 is so that we can work around the dataframe table write limitation with dataframe create table.
* Later when we refresh trips_table, it will still show the correct data.

In [23]:
spark.sql(
"""
CREATE TABLE fare_adjustment (ts BIGINT, uuid STRING, rider STRING, driver STRING, fare DOUBLE, city STRING) 
USING HUDI
LOCATION "/tmp/hudicow/fare_adjustment"
;
"""
)

res23: DataFrame = []

In [24]:
spark.sql(
"""
INSERT INTO fare_adjustment VALUES 
(1695091554788,'e96c4396-3fad-413a-a942-4cb36106d721','rider-C','driver-M',-2.70 ,'san_francisco'),
(1695530237068,'3f3d9565-7261-40e6-9b39-b8aa784f95e2','rider-K','driver-U',64.20 ,'san_francisco'),
(1695241330902,'ea4c36ff-2069-4148-9927-ef8c1a5abd24','rider-H','driver-R',66.60 ,'sao_paulo'    ),
(1695115999911,'c8abbe79-8d89-47ea-b4ce-4d224bae5bfa','rider-J','driver-T',1.85,'chennai'      );
"""
)

res24: DataFrame = []

In [25]:
// Register the dataframe created table to Hive metastore.
spark.sql(
s"""
CREATE TABLE IF NOT EXISTS trips_table2 (
    ts BIGINT,
    uuid STRING,
    rider STRING,
    driver STRING,
    fare DOUBLE,
    city STRING
) USING HUDI
PARTITIONED BY (city)
LOCATION "$basePath"
"""
)

res25: DataFrame = []

In [26]:
spark.sql(
s"""
SELECT * FROM trips_table2
"""
).show()

+-------------------+--------------------+--------------------+----------------------+--------------------+-------------+--------------------+-------+--------+-----+-------------+
|_hoodie_commit_time|_hoodie_commit_seqno|  _hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|           ts|                uuid|  rider|  driver| fare|         city|
+-------------------+--------------------+--------------------+----------------------+--------------------+-------------+--------------------+-------+--------+-----+-------------+
|  20250305202842976|20250305202842976...|20250305202833534...|    city=san_francisco|ce80064b-d0eb-474...|1695159649087|334e26e9-8355-45c...|rider-A|driver-K|210.1|san_francisco|
|  20250305202842976|20250305202842976...|20250305202833534...|    city=san_francisco|ce80064b-d0eb-474...|1695091554788|e96c4396-3fad-413...|rider-C|driver-M|304.7|san_francisco|
|  20250305202839238|20250305202839238...|20250305202833534...|    city=san_francisco|ce80064b-d0eb-

In [27]:
spark.sql(
s"""
MERGE INTO trips_table2 AS target
USING fare_adjustment AS source
ON target.uuid = source.uuid
WHEN MATCHED THEN UPDATE SET fare = source.fare
WHEN NOT MATCHED THEN INSERT *
;
"""
)

res27: DataFrame = []

In [28]:
spark.catalog.refreshTable("trips_table")
spark.sql(
s"""
SELECT * FROM trips_table
"""
).show()

+-------------------+--------------------+--------------------+----------------------+--------------------+-------------+--------------------+-------+--------+-----+-------------+
|_hoodie_commit_time|_hoodie_commit_seqno|  _hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|           ts|                uuid|  rider|  driver| fare|         city|
+-------------------+--------------------+--------------------+----------------------+--------------------+-------------+--------------------+-------+--------+-----+-------------+
|  20250305202842976|20250305202842976...|20250305202833534...|    city=san_francisco|ce80064b-d0eb-474...|1695159649087|334e26e9-8355-45c...|rider-A|driver-K|210.1|san_francisco|
|  20250305202846930|20250305202846930...|20250305202833534...|    city=san_francisco|ce80064b-d0eb-474...|1695091554788|e96c4396-3fad-413...|rider-C|driver-M| -2.7|san_francisco|
|  20250305202839238|20250305202839238...|20250305202833534...|    city=san_francisco|ce80064b-d0eb-

In [29]:
// Or use spark load.
spark.read.format("Hudi").load(basePath).show()

+-------------------+--------------------+--------------------+----------------------+--------------------+-------------+--------------------+-------+--------+-----+-------------+
|_hoodie_commit_time|_hoodie_commit_seqno|  _hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|           ts|                uuid|  rider|  driver| fare|         city|
+-------------------+--------------------+--------------------+----------------------+--------------------+-------------+--------------------+-------+--------+-----+-------------+
|  20250305202842976|20250305202842976...|20250305202833534...|    city=san_francisco|ce80064b-d0eb-474...|1695159649087|334e26e9-8355-45c...|rider-A|driver-K|210.1|san_francisco|
|  20250305202846930|20250305202846930...|20250305202833534...|    city=san_francisco|ce80064b-d0eb-474...|1695091554788|e96c4396-3fad-413...|rider-C|driver-M| -2.7|san_francisco|
|  20250305202839238|20250305202839238...|20250305202833534...|    city=san_francisco|ce80064b-d0eb-

* If we check the parquet files, now we have the fourth parquet files.

In [30]:
val command = """find /tmp/hudicow/trips_table -path "*san_francisco*" -name " *.parquet" -exec stat -c \"%y %s %n\" {} + | sort"""
lazy val result = Seq("bash", "-c", s"eval $command").!!
println(result)

2025-03-05 20:28:36.456832582 +0900 436116 /tmp/hudicow/trips_table/city=san_francisco/ce80064b-d0eb-474f-adf5-5e18721902da-0_0-6-0_20250305202833534.parquet
2025-03-05 20:28:39.815689984 +0900 436091 /tmp/hudicow/trips_table/city=san_francisco/ce80064b-d0eb-474f-adf5-5e18721902da-0_0-20-22_20250305202839238.parquet
2025-03-05 20:28:43.476375396 +0900 436094 /tmp/hudicow/trips_table/city=san_francisco/ce80064b-d0eb-474f-adf5-5e18721902da-0_0-47-47_20250305202842976.parquet
2025-03-05 20:28:47.487685057 +0900 436223 /tmp/hudicow/trips_table/city=san_francisco/ce80064b-d0eb-474f-adf5-5e18721902da-0_0-99-99_20250305202846930.parquet



command: String = "find /tmp/hudicow/trips_table -path \"*san_francisco*\" -name \" *.parquet\" -exec stat -c \\\"%y %s %n\\\" {} + | sort"
result: String = [lazy]

In [31]:
val command = """find /tmp/hudicow/trips_table -name "*.commit" -exec stat -c \"%y %s %n\" {} + | sort"""
lazy val result = Seq("bash", "-c", s"eval $command").!!
lazy val hudiMetas = result.split("\n")
for(hudiMeta <- hudiMetas){
    val path = hudiMeta.split(" ")(4)
    println(path)
    spark.read.format("avro").load(path).select("operationType", "partitionToWriteStats", "extraMetadata", "version").show(truncate=false)
    println()
}

/tmp/hudicow/trips_table/.hoodie/timeline/20250305202833534_20250305202836772.commit
+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------

command: String = "find /tmp/hudicow/trips_table -name \"*.commit\" -exec stat -c \\\"%y %s %n\\\" {} + | sort"
result: String = [lazy]
hudiMetas: Array[String] = [lazy]

### Delete data

In [32]:
// Lets  delete rider: rider-D
val deletesDF = spark.read.format("hudi").load(basePath).filter($"rider" === "rider-F")

deletesDF.write.format("hudi").
  option("hoodie.datasource.write.operation", "delete").
  option("hoodie.datasource.write.partitionpath.field", "city").
  option("hoodie.table.name", tableName).
  mode(Append).
  save(basePath)

deletesDF: Dataset[Row] = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 9 more fields]

In [33]:
spark.read.format("hudi").load(basePath).show()

+-------------------+--------------------+--------------------+----------------------+--------------------+-------------+--------------------+-------+--------+-----+-------------+
|_hoodie_commit_time|_hoodie_commit_seqno|  _hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|           ts|                uuid|  rider|  driver| fare|         city|
+-------------------+--------------------+--------------------+----------------------+--------------------+-------------+--------------------+-------+--------+-----+-------------+
|  20250305202842976|20250305202842976...|20250305202833534...|    city=san_francisco|ce80064b-d0eb-474...|1695159649087|334e26e9-8355-45c...|rider-A|driver-K|210.1|san_francisco|
|  20250305202846930|20250305202846930...|20250305202833534...|    city=san_francisco|ce80064b-d0eb-474...|1695091554788|e96c4396-3fad-413...|rider-C|driver-M| -2.7|san_francisco|
|  20250305202839238|20250305202839238...|20250305202833534...|    city=san_francisco|ce80064b-d0eb-

* If we check sao_paulo, we have 3 files now.
    * 1 original.
    * 1 when we do merge upsert [Merging Data (Partial Updates)](###-Merging-Data-(Partial-Updates)-if-use-sql-and-not-CustomMergeIntoConnector.)
    * 1 from delete.

In [34]:
val command = """find /tmp/hudicow/trips_table -path "*sao_paulo*" -name " *.parquet" -exec stat -c \"%y %s %n\" {} + | sort"""
lazy val result = Seq("bash", "-c", s"eval $command").!!
println(result)

2025-03-05 20:28:36.456832582 +0900 435948 /tmp/hudicow/trips_table/city=sao_paulo/ce80064b-d0eb-474f-adf5-5e18721902da-1_0-6-0_20250305202833534.parquet
2025-03-05 20:28:47.608722516 +0900 436000 /tmp/hudicow/trips_table/city=sao_paulo/ce80064b-d0eb-474f-adf5-5e18721902da-1_2-99-101_20250305202846930.parquet
2025-03-05 20:28:49.424791996 +0900 436016 /tmp/hudicow/trips_table/city=sao_paulo/ce80064b-d0eb-474f-adf5-5e18721902da-1_0-118-124_20250305202849026.parquet



command: String = "find /tmp/hudicow/trips_table -path \"*sao_paulo*\" -name \" *.parquet\" -exec stat -c \\\"%y %s %n\\\" {} + | sort"
result: String = [lazy]

* Lets check the commit files.

In [35]:
val command = """find /tmp/hudicow/trips_table -name "*.commit" -exec stat -c \"%y %s %n\" {} + | sort | tail -n 2"""
lazy val result = Seq("bash", "-c", s"eval $command").!!
lazy val hudiMetas = result.split("\n")
for(hudiMeta <- hudiMetas){
    val path = hudiMeta.split(" ")(4)
    println(path)
    spark.read.format("avro").load(path).select("operationType", "partitionToWriteStats", "extraMetadata", "version").show(truncate=false)
    println()
}

/tmp/hudicow/trips_table/.hoodie/timeline/20250305202846930_20250305202847840.commit
+-------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------

command: String = "find /tmp/hudicow/trips_table -name \"*.commit\" -exec stat -c \\\"%y %s %n\\\" {} + | sort | tail -n 2"
result: String = [lazy]
hudiMetas: Array[String] = [lazy]

* Now we can see "DELETE_PREPPED" operation.

In [36]:
val command = """find /tmp/hudicow/trips_table -path "*sao_paulo*" -name "*.parquet" -exec stat -c \"%y %s %n\" {} + | sort"""
lazy val result = Seq("bash", "-c", s"eval $command").!!
lazy val parquetFiles = result.split("\n")
for(parquetFile <- parquetFiles){
    val path = parquetFile.split(" ")(4)
    println(path)
    spark.read.format("parquet").load(path).show(truncate=false)
    println()
}

/tmp/hudicow/trips_table/city=sao_paulo/ce80064b-d0eb-474f-adf5-5e18721902da-1_0-6-0_20250305202833534.parquet
+-------------------+---------------------+---------------------+----------------------+----------------------------------------------------------------------+-------------+------------------------------------+-------+--------+-----+---------+
|_hoodie_commit_time|_hoodie_commit_seqno |_hoodie_record_key   |_hoodie_partition_path|_hoodie_file_name                                                     |ts           |uuid                                |rider  |driver  |fare |city     |
+-------------------+---------------------+---------------------+----------------------+----------------------------------------------------------------------+-------------+------------------------------------+-------+--------+-----+---------+
|20250305202833534  |20250305202833534_0_4|20250305202833534_0_3|city=sao_paulo        |ce80064b-d0eb-474f-adf5-5e18721902da-1_0-6-0_20250305202833534.parque

command: String = "find /tmp/hudicow/trips_table -path \"*sao_paulo*\" -name \"*.parquet\" -exec stat -c \\\"%y %s %n\\\" {} + | sort"
result: String = [lazy]
parquetFiles: Array[String] = [lazy]

* And the following is our final dir structure.

In [37]:
lazy val result = "tree -a /tmp/hudicow/trips_table".!!
println(result)

/tmp/hudicow/trips_table
├── .hoodie
│   ├── .aux
│   │   └── .bootstrap
│   │       ├── .fileids
│   │       └── .partitions
│   ├── .hoodie.properties.crc
│   ├── .schema
│   ├── .temp
│   ├── hoodie.properties
│   ├── metadata
│   │   ├── .hoodie
│   │   │   ├── .aux
│   │   │   │   └── .bootstrap
│   │   │   │       ├── .fileids
│   │   │   │       └── .partitions
│   │   │   ├── .hoodie.properties.crc
│   │   │   ├── .schema
│   │   │   ├── .temp
│   │   │   ├── hoodie.properties
│   │   │   └── timeline
│   │   │       ├── .00000000000000000.deltacommit.inflight.crc
│   │   │       ├── .00000000000000000.deltacommit.requested.crc
│   │   │       ├── .00000000000000000_20250305202834701.deltacommit.crc
│   │   │       ├── .20250305202833534.deltacommit.inflight.crc
│   │   │       ├── .20250305202833534.deltacommit.requested.crc
│   │   │       ├── .20250305202833534_20250305202836740.deltacommit.crc
│   │   │       ├── .20250305202839238.deltacommit.inflight.crc
│   │   │       ├

result: String = [lazy]

In [38]:
spark.stop()